In [ ]:
!unzip -q /content/drive/MyDrive/Kaggle/Foursquare/data/foursquare-location-matching.zip

In [ ]:
!pip install -q Levenshtein
!pip install lightgbm --upgrade
!pip install  lightgbm==3.3.1

     |████████████████████████████████| 194 kB 15.1 MB/s 
     |████████████████████████████████| 2.0 MB 91.0 MB/s 
     |████████████████████████████████| 104 kB 66.9 MB/s 
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 2.0 MB 14.1 MB/s 
  Attempting uninstall: lightgbm
    Found existing installation: lightgbm 2.2.3
    Uninstalling lightgbm-2.2.3:
      Successfully uninstalled lightgbm-2.2.3
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 2.0 MB 14.4 MB/s 
  Attempting uninstall: lightgbm
    Found existing installation: lightgbm 3.3.2
    Uninstalling lightgbm-3.3.2:
      Successfully uninstalled lightgbm-3.3.2


In [ ]:
import pandas as pd
import numpy as np
from sklearn.neighbors import BallTree
from sklearn.neighbors import KNeighborsRegressor
from tqdm.notebook import tqdm
from sklearn.model_selection import GroupKFold, KFold, StratifiedKFold
import pickle
import lightgbm as lgbm
import Levenshtein
import difflib
import sklearn
import joblib
from sklearn.metrics import f1_score,accuracy_score,recall_score

In [ ]:
def reduce_mem_usage(df):
    """ iterate through all the columns of a dataframe and modify the data type
        to reduce memory usage.        
    """
    start_mem = df.memory_usage().sum() / 1024**2
    print('Memory usage of dataframe is {:.2f} MB'.format(start_mem))
    
    for col in df.columns:
        col_type = df[col].dtype
        
        if col_type != object:
            c_min = df[col].min()
            c_max = df[col].max()
            if str(col_type)[:3] == 'int':
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)  
            else:
                if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                    df[col] = df[col].astype(np.float16)
                elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)

    end_mem = df.memory_usage().sum() / 1024**2
    print('Memory usage after optimization is: {:.2f} MB'.format(end_mem))
    print('Decreased by {:.1f}%'.format(100 * (start_mem - end_mem) / start_mem))
    
    return df

In [ ]:
input_path = '/content/drive/MyDrive/Kaggle/Foursquare/data/newdata/'
import gc
print('start read csv 0')
train = pd.read_csv(input_path + 'train_data_v8_0.csv')
train = reduce_mem_usage(train)
train['fold'] = 0

print('start read csv 1')
train_1 = pd.read_csv(input_path + 'train_data_v8_1.csv')
train_1 = reduce_mem_usage(train_1)
train_1['fold'] = 1
train = pd.concat([train,train_1],axis=0)
del train_1
gc.collect()

print('start read csv 2')
train_2 = pd.read_csv(input_path + 'train_data_v8_2.csv')
train_2 = reduce_mem_usage(train_2)
train_2['fold'] = 2
train = pd.concat([train,train_2],axis=0)
del train_2
gc.collect()

print('start read csv 3')
train_3 = pd.read_csv(input_path + 'train_data_v8_3.csv')
train_3 = reduce_mem_usage(train_3)
train_3['fold'] = 3
train = pd.concat([train,train_3],axis=0)
del train_3
gc.collect()

print('start read csv 4')
train_4 = pd.read_csv(input_path + 'train_data_v8_4.csv')
train_4 = reduce_mem_usage(train_4)
train_4['fold'] = 4
train = pd.concat([train,train_4],axis=0)
del train_4
gc.collect()

start read csv 0


/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2882: DtypeWarning: Columns (91,94,104) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


Memory usage of dataframe is 1744.70 MB
Memory usage after optimization is: 643.88 MB
Decreased by 63.1%
start read csv 1
Memory usage of dataframe is 1745.04 MB
Memory usage after optimization is: 646.08 MB
Decreased by 63.0%
start read csv 2


/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2882: DtypeWarning: Columns (94,104) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


Memory usage of dataframe is 1744.30 MB
Memory usage after optimization is: 643.73 MB
Decreased by 63.1%
start read csv 3
Memory usage of dataframe is 1745.01 MB
Memory usage after optimization is: 637.76 MB
Decreased by 63.5%
start read csv 4


/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2882: DtypeWarning: Columns (94,101,104) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


Memory usage of dataframe is 1745.20 MB
Memory usage after optimization is: 646.14 MB
Decreased by 63.0%


0

In [ ]:
train = train.reset_index(drop=True)

In [ ]:
train.shape

(10890523, 106)

In [ ]:
tdata = pd.read_csv('./train.csv')
tdata['name'] = tdata['name'].isna().astype(int)
tdata['categories'] = tdata['categories'].isna().astype(int)
tdata['address'] = tdata['address'].isna().astype(int)
tdata['state'] = tdata['state'].isna().astype(int)
tdata['url'] = tdata['url'].isna().astype(int)
tdata['country'] = tdata['country'].isna().astype(int)
tdata['phone'] = tdata['phone'].isna().astype(int)

tdata = tdata[['id','name', 'categories', 'address', 'state', 
               'url', 'country','phone']].drop_duplicates().reset_index(drop=True)

In [ ]:
def manhattan(lat1, long1, lat2, long2):
    return np.abs(lat2 - lat1) + np.abs(long2 - long1)

# get haversine distance
def vectorized_haversine(lats1, lats2, longs1, longs2):
    radius = 6371
    dlat=np.radians(lats2 - lats1)
    dlon=np.radians(longs2 - longs1)
    a = np.sin(dlat/2) * np.sin(dlat/2) + np.cos(np.radians(lats1)) \
        * np.cos(np.radians(lats2)) * np.sin(dlon/2) * np.sin(dlon/2)
    c = 2 * np.arctan2(np.sqrt(a), np.sqrt(1-a))
    d = radius * c
    return d

def add_features(train,tdata):
  train['latdiff'] = train['latitude_x'] - train['latitude_y']
  train['londiff'] = train['longitude_x'] - train['longitude_y']
  train['manhattan'] = manhattan(train['latitude_x'], train['longitude_x'], train['latitude_y'], train['longitude_y'])
  train['euclidean'] = (train['latitude_x'] ** 2 + train['longitude_x'] ** 2) ** 0.5
  train['haversine'] = vectorized_haversine(train['latitude_x'], train['longitude_x'], train['latitude_y'], train['longitude_y'])
  train['kdist_diff'] = (train['kdist'] - train['kdist_country']) / train['kdist_country']
  train['kneighbors_mean'] = train[['kneighbors', 'kneighbors_country']].mean(axis = 1)
  train['sim_sum'] = train[[col for col in train.columns if 'sim' in col]].sum(axis=1)
  train['gesh_sum'] = train[[col for col in train.columns if 'gesh' in col]].sum(axis=1)
  train['leven_sum'] = train[[col for col in train.columns if '_leven' in col]].sum(axis=1)
  train['jaro_sum'] = train[[col for col in train.columns if 'jaro' in col]].sum(axis=1)
  train['lcs_sum'] = train[[col for col in train.columns if '_lcs' in col]].sum(axis=1)

  train['sim_std'] = train[[col for col in train.columns if 'sim' in col]].std(axis=1)
  train['gesh_std'] = train[[col for col in train.columns if 'gesh' in col]].std(axis=1)
  train['leven_std'] = train[[col for col in train.columns if '_leven' in col]].std(axis=1)
  train['jaro_std'] = train[[col for col in train.columns if 'jaro' in col]].std(axis=1)
  train['lcs_std'] = train[[col for col in train.columns if '_lcs' in col]].std(axis=1)

  train = train.merge(tdata, on='id', how='left')
  train = train.merge(tdata, left_on='near_id', right_on='id', how='left', suffixes=['_1','_2'])
  train = train.drop('id_2', axis=1).rename(columns={'id_1':'id'})
  
  train['info_power_1'] = train[[col for col in train.columns if '_1' in col]].lt(1).sum(axis=1)
  train['info_power_2'] = train[[col for col in train.columns if '_2' in col]].lt(1).sum(axis=1)
  train['info_diff'] = train['info_power_1'] - train['info_power_2']
  del tdata
    
  return train

In [ ]:
train = add_features(train,tdata)

In [ ]:
train = reduce_mem_usage(train)

Memory usage of dataframe is 5244.94 MB
Memory usage after optimization is: 3915.53 MB
Decreased by 25.3%


In [ ]:
import gc
gc.collect()

50

In [ ]:
features = []
num_features = ['kdist', 'kneighbors', 'kdist_country', 'kneighbors_country', 
       'name_sim','name_distilbert-base-uncased_cat_vec_sim', 'name_gesh', 'name_leven',
       'name_jaro', 'name_lcs', 'name_len_diff', 'name_nleven', 'name_nlcsk',
       'name_nlcs', 'address_sim', 'address_distilbert-base-uncased_cat_vec_sim',
       'address_gesh', 'address_leven', 'address_jaro', 'address_lcs',
       'address_len_diff', 'address_nleven', 'address_nlcsk', 'address_nlcs',
       'city_gesh', 'city_leven', 'city_jaro', 'city_lcs', 'city_len_diff',
       'city_nleven', 'city_nlcsk', 'city_nlcs', 'state_sim',
       'state_distilbert-base-uncased_cat_vec_sim', 'state_gesh', 'state_leven',
       'state_jaro', 'state_lcs', 'state_len_diff', 'state_nleven',
       'state_nlcsk', 'state_nlcs', 'zip_gesh', 'zip_leven', 'zip_jaro',
       'zip_lcs', 'url_sim','url_distilbert-base-uncased_cat_vec_sim', 'url_gesh', 'url_leven',
       'url_jaro', 'url_lcs', 'url_len_diff', 'url_nleven', 'url_nlcsk',
       'url_nlcs', 'phone_sim', 'phone_distilbert-base-uncased_cat_vec_sim',
       'phone_gesh', 'phone_leven', 'phone_jaro', 'phone_lcs',
       'categories_sim', 'categories_distilbert-base-uncased_cat_vec_sim',
       'categories_gesh', 'categories_leven', 'categories_jaro',
       'categories_lcs', 'categories_len_diff', 'categories_nleven',
       'categories_nlcsk', 'categories_nlcs', 'country_sim',
       'country_distilbert-base-uncased_cat_vec_sim', 'country_gesh',
       'country_leven', 'country_jaro', 'country_lcs', 'country_len_diff',
       'country_nleven', 'country_nlcsk', 'country_nlcs','latitude_y',
       'longitude_y', 'latdiff', 'londiff', 'manhattan', 'euclidean', 'haversine',
       'kdist_diff', 'kneighbors_mean', 'sim_sum', 'gesh_sum', 'leven_sum',
       'jaro_sum', 'lcs_sum', 'sim_std', 'gesh_std', 'leven_std', 'jaro_std',
       'lcs_std', 'name_1', 'categories_1', 'address_1', 'state_1', 'url_1',
       'country_1', 'name_2', 'categories_2', 'address_2', 'state_2', 'url_2',
       'country_2', 'info_power_1', 'info_power_2', 'info_diff']

cat_features = [ 'city_x', 'city_y', 'country_x', 'country_y',  'state_x', 'state_y' , 'categories_x','categories_y']
train[cat_features] = train[cat_features].astype('category')

features = num_features  + cat_features

In [ ]:
import lightgbm as lgbm
import pickle
import joblib
import gc

params = {
    'learning_rate': 0.05,
    'reg_alpha': 0.1,
    'reg_lambda': 0.1,
    'random_state': 42,
    'max_depth': 3000,   
    'num_leaves': 1500, 
    'n_estimators': 100000, 
    "colsample_bytree": 0.9,
}

def fit_lgbm(train):
    models = []
    for fold in tqdm(range(0,5)):
          print(f'==fold{fold}==')
          train_data = train[train['fold'] != fold]
          X_train , y_train = train_data[features] , train_data['label']
          val_data =  train[train['fold'] == fold]
          X_val , y_val = val_data[features] , val_data['label']
          del train_data , val_data
          for _ in range(5):
            gc.collect()
          model = lgbm.LGBMClassifier(**params)
          model.fit(X_train, y_train, eval_set=[(X_val, y_val)],early_stopping_rounds=10, eval_metric=['logloss','auc'],verbose=30)
          pred = model.predict(X_val)
          print('accuracy',accuracy_score(y_val.values,pred) , 'recall',recall_score(y_val.values,pred),'f1',f1_score(y_val.values,pred))
          models.append(model)
          joblib.dump(model,f'/content/drive/MyDrive/Kaggle/Foursquare/model/lgbm_fold{fold}_v7.pkl')
          del  X_train , y_train , X_val , y_val ,pred
          gc.collect()
    return models


In [ ]:
import gc
gc.collect()

62

In [ ]:
train['label'].value_counts()

0    10220397
1      670126
Name: label, dtype: int64

In [ ]:
gc.collect()

141

In [ ]:
models = fit_lgbm(train)

  0%|          | 0/5 [00:00<?, ?it/s]

==fold0==


/usr/local/lib/python3.7/dist-packages/lightgbm/sklearn.py:726: UserWarning: 'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. Pass 'early_stopping()' callback via 'callbacks' argument instead.
  _log_warning("'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. "
/usr/local/lib/python3.7/dist-packages/lightgbm/sklearn.py:736: UserWarning: 'verbose' argument is deprecated and will be removed in a future release of LightGBM. Pass 'log_evaluation()' callback via 'callbacks' argument instead.
  _log_warning("'verbose' argument is deprecated and will be removed in a future release of LightGBM. "
/usr/local/lib/python3.7/dist-packages/lightgbm/basic.py:1780: UserWarning: Overriding the parameters from Reference Dataset.
  _log_warning('Overriding the parameters from Reference Dataset.')
/usr/local/lib/python3.7/dist-packages/lightgbm/basic.py:1513: UserWarning: categorical_column in param dict i

[30]	valid_0's binary_logloss: 0.0825176	valid_0's auc: 0.982191
[60]	valid_0's binary_logloss: 0.0672756	valid_0's auc: 0.984389
[90]	valid_0's binary_logloss: 0.0635169	valid_0's auc: 0.985546
[120]	valid_0's binary_logloss: 0.0623739	valid_0's auc: 0.986208
[150]	valid_0's binary_logloss: 0.0620641	valid_0's auc: 0.986511
accuracy 0.977811826621492 recall 0.7532694635837804 f1 0.8066421254801537
==fold1==


/usr/local/lib/python3.7/dist-packages/lightgbm/sklearn.py:726: UserWarning: 'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. Pass 'early_stopping()' callback via 'callbacks' argument instead.
  _log_warning("'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. "
/usr/local/lib/python3.7/dist-packages/lightgbm/sklearn.py:736: UserWarning: 'verbose' argument is deprecated and will be removed in a future release of LightGBM. Pass 'log_evaluation()' callback via 'callbacks' argument instead.
  _log_warning("'verbose' argument is deprecated and will be removed in a future release of LightGBM. "
/usr/local/lib/python3.7/dist-packages/lightgbm/basic.py:1780: UserWarning: Overriding the parameters from Reference Dataset.
  _log_warning('Overriding the parameters from Reference Dataset.')
/usr/local/lib/python3.7/dist-packages/lightgbm/basic.py:1513: UserWarning: categorical_column in param dict i

...

[30]	valid_0's binary_logloss: 0.0826442	valid_0's auc: 0.981552
[60]	valid_0's binary_logloss: 0.0674292	valid_0's auc: 0.983397
[90]	valid_0's binary_logloss: 0.0638201	valid_0's auc: 0.98459
[120]	valid_0's binary_logloss: 0.0626829	valid_0's auc: 0.985271
[150]	valid_0's binary_logloss: 0.0624053	valid_0's auc: 0.985617
accuracy 0.9778752019016993 recall 0.7543341389908393 f1 0.807619351875886
==fold3==


/usr/local/lib/python3.7/dist-packages/lightgbm/sklearn.py:726: UserWarning: 'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. Pass 'early_stopping()' callback via 'callbacks' argument instead.
  _log_warning("'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. "
/usr/local/lib/python3.7/dist-packages/lightgbm/sklearn.py:736: UserWarning: 'verbose' argument is deprecated and will be removed in a future release of LightGBM. Pass 'log_evaluation()' callback via 'callbacks' argument instead.
  _log_warning("'verbose' argument is deprecated and will be removed in a future release of LightGBM. "
/usr/local/lib/python3.7/dist-packages/lightgbm/basic.py:1780: UserWarning: Overriding the parameters from Reference Dataset.
  _log_warning('Overriding the parameters from Reference Dataset.')
/usr/local/lib/python3.7/dist-packages/lightgbm/basic.py:1513: UserWarning: categorical_column in param dict i

[30]	valid_0's binary_logloss: 0.0819687	valid_0's auc: 0.982497
[60]	valid_0's binary_logloss: 0.0668036	valid_0's auc: 0.984178
[90]	valid_0's binary_logloss: 0.0630712	valid_0's auc: 0.985421
[120]	valid_0's binary_logloss: 0.0619167	valid_0's auc: 0.986115
[150]	valid_0's binary_logloss: 0.0616141	valid_0's auc: 0.986412
accuracy 0.9779782500214617 recall 0.7610993186777301 f1 0.8104462041822751
==fold4==


/usr/local/lib/python3.7/dist-packages/lightgbm/sklearn.py:726: UserWarning: 'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. Pass 'early_stopping()' callback via 'callbacks' argument instead.
  _log_warning("'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. "
/usr/local/lib/python3.7/dist-packages/lightgbm/sklearn.py:736: UserWarning: 'verbose' argument is deprecated and will be removed in a future release of LightGBM. Pass 'log_evaluation()' callback via 'callbacks' argument instead.
  _log_warning("'verbose' argument is deprecated and will be removed in a future release of LightGBM. "
/usr/local/lib/python3.7/dist-packages/lightgbm/basic.py:1780: UserWarning: Overriding the parameters from Reference Dataset.
  _log_warning('Overriding the parameters from Reference Dataset.')
/usr/local/lib/python3.7/dist-packages/lightgbm/basic.py:1513: UserWarning: categorical_column in param dict i

[30]	valid_0's binary_logloss: 0.0820514	valid_0's auc: 0.982573
[60]	valid_0's binary_logloss: 0.0667541	valid_0's auc: 0.984497
[90]	valid_0's binary_logloss: 0.0630655	valid_0's auc: 0.985628
[120]	valid_0's binary_logloss: 0.0618481	valid_0's auc: 0.986364
[150]	valid_0's binary_logloss: 0.061492	valid_0's auc: 0.986769
accuracy 0.9778067369766229 recall 0.7532195558205396 f1 0.8069769205894213


In [ ]:
def valid_lgbm(train,thesh=0.5):
  train['pred'] = -1
  for fold in tqdm(range(5)):
    print(f'==fold{fold}==') 
    X_val , y_val = train[train['fold'] == fold][features] , train[train['fold'] == fold]['label']
    model = joblib.load(f'/content/drive/MyDrive/Kaggle/Foursquare/model/lgbm_fold{fold}_v7.pkl')
#     model = joblib.load(f"../input/fork-of-train-and-submit-model/lgbm_fold{fold}.pkl")
    pred = model.predict_proba(X_val)[:,1]
    train.loc[train['fold'] == fold , 'pred'] = pred
    train['pred'] = [1 if i > hthes else 0 for i in train['pred']]
  print('accuracy',accuracy_score(train['label'].values,train['pred'].values) , 'recall',recall_score(train['label'].values,train['pred'].values),'f1',f1_score(train['label'].values,train['pred'].values))
  return train 

In [ ]:
result = train[train['pred'] == 1][~train['near_id'].isnull()].sort_values(by='id_x')[['id_x','near_id']]
result = result.reset_index(drop=True)
result = result.groupby('id_x')['near_id'].apply(list).reset_index()
result.columns = ['id_x','matches'] 
result['matches'] = result.apply(lambda x : [x['id_x']] + x['matches'],axis=1)

In [ ]:
result

In [ ]:
train = pd.read_csv('./train.csv')
train_copy = train.copy()
train['matchs'] = ''
poi_id = train_copy.groupby('point_of_interest')['id'].apply(list)
poi_id = poi_id.reset_index()
poi_id.columns = ['point_of_interest','target']
train = train.merge(poi_id,on='point_of_interest')
train = train.merge(result,left_on='id',right_on='id_x',how='left')
train.loc[train['matches'].isnull() , 'matches'] = [i.split(',') for i in train.loc[train['matches'].isnull() , 'id']]
train['matches'] = [i.split(',') if isinstance(i,str) else i for i in train['matches']]

In [ ]:
train['matches'] = train['matches'].apply(set).apply(list)

In [ ]:
def get_score(input_df,col):
    input_df = input_df[~input_df[col].isnull()]
    scores = []
    for targets, preds in zip(input_df['target'].to_numpy(), input_df[col].to_numpy()):
        score = len((set(targets) & set(preds))) / len((set(targets) | set(preds)))
        scores.append(score)
    scores = np.array(scores)
    return scores.mean()

get_score(train,'matches')

In [ ]:
def postprocess_train(df):
    id2match = dict(zip(df["id"].values, df["matches"]))
    for match in tqdm(df["matches"]):
        if  len(match) == 1:        
            continue
        base = match[0]
        for m in match[1:]:
            if not base in id2match[m]:
                id2match[m].append(base)
    df["matches"] = df["id"].map(id2match)
    return df 

In [ ]:
train = postprocess_train(train)
get_score(train,'matches')

In [ ]:
import gc
del train
gc.collect()